In [24]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Rio De Janeiro - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [5]:
data = pd.read_csv('2003_01_model_input_RJ.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t),Rio De Janeiro - Consumo de Cimento (t)
0,2003,0.773489,3.641734,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,...,18.036745,66.777512,0.959633,8.356512,3.583430e+08,1.813534e+07,22.143720,3.472846e+08,2377.549731,3021.269
1,2004,0.774193,3.879151,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,...,17.373589,70.098960,0.962531,8.282350,3.631565e+08,1.820415e+07,22.209810,3.487366e+08,2400.424606,3092.530
2,2005,0.774914,3.969821,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,...,16.298207,73.386822,0.949820,8.259763,3.674546e+08,1.823716e+07,22.259791,3.497876e+08,2511.833086,3324.329
3,2006,0.775653,3.958649,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,...,18.160850,75.523920,0.945490,8.165392,3.711498e+08,1.831286e+07,22.286909,3.503984e+08,2676.431443,3665.682
4,2007,0.776412,3.899249,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,...,18.679514,78.858203,0.869522,8.139058,3.742425e+08,1.841057e+07,22.295610,3.506279e+08,2858.554649,4107.097
5,2008,0.777194,3.862900,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,...,19.041313,80.733596,0.819193,8.106163,3.769087e+08,1.848763e+07,22.305022,3.507645e+08,3010.012911,4300.676
6,2009,0.778002,3.934097,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,...,19.160541,81.195790,0.873595,8.134661,3.784121e+08,1.857076e+07,22.275112,3.502037e+08,3108.577595,3966.930
7,2010,0.778838,4.053038,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,...,18.952421,81.281067,0.913459,8.098147,3.868458e+08,1.857479e+07,22.286297,3.512210e+08,3181.826392,4199.911
8,2011,0.779704,4.190909,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,...,18.939670,81.492721,0.874916,8.084479,3.942901e+08,1.848968e+07,22.267033,3.516849e+08,3209.638424,4390.254
9,2012,0.780264,4.443161,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,...,18.924341,80.811760,0.826735,9.891336,4.002860e+08,1.838631e+07,22.204155,3.513894e+08,3199.169780,4733.632


In [6]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
0,0.773489,3.641734,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,0.895303,17.414471,...,26.869869,18.036745,66.777512,0.959633,8.356512,3.583430e+08,1.813534e+07,22.143720,3.472846e+08,2377.549731
1,0.774193,3.879151,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,0.746570,18.346431,...,25.953791,17.373589,70.098960,0.962531,8.282350,3.631565e+08,1.820415e+07,22.209810,3.487366e+08,2400.424606
2,0.774914,3.969821,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,0.589811,18.939013,...,24.741422,16.298207,73.386822,0.949820,8.259763,3.674546e+08,1.823716e+07,22.259791,3.497876e+08,2511.833086
3,0.775653,3.958649,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,0.606044,22.899961,...,24.047847,18.160850,75.523920,0.945490,8.165392,3.711498e+08,1.831286e+07,22.286909,3.503984e+08,2676.431443
4,0.776412,3.899249,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,0.633401,31.018587,...,23.335065,18.679514,78.858203,0.869522,8.139058,3.742425e+08,1.841057e+07,22.295610,3.506279e+08,2858.554649
5,0.777194,3.862900,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,0.587619,36.697984,...,23.145598,19.041313,80.733596,0.819193,8.106163,3.769087e+08,1.848763e+07,22.305022,3.507645e+08,3010.012911
6,0.778002,3.934097,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,0.566759,32.933440,...,23.101387,19.160541,81.195790,0.873595,8.134661,3.784121e+08,1.857076e+07,22.275112,3.502037e+08,3108.577595
7,0.778838,4.053038,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,0.593537,29.177236,...,22.850128,18.952421,81.281067,0.913459,8.098147,3.868458e+08,1.857479e+07,22.286297,3.512210e+08,3181.826392
8,0.779704,4.190909,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,0.600821,25.153610,...,22.787473,18.939670,81.492721,0.874916,8.084479,3.942901e+08,1.848968e+07,22.267033,3.516849e+08,3209.638424
9,0.780264,4.443161,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,0.604731,23.695979,...,22.961436,18.924341,80.811760,0.826735,9.891336,4.002860e+08,1.838631e+07,22.204155,3.513894e+08,3199.169780


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     3092.530
1     3324.329
2     3665.682
3     4107.097
4     4300.676
5     3966.930
6     4199.911
7     4390.254
8     4733.632
9     4864.477
10    4454.468
11    3639.072
12    2811.648
13    2284.918
14    2391.212
15    2715.843
16    2935.607
17    2829.682
18    2682.995
19         NaN
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
0,-1.659894,-1.327573,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,2.670619,-1.465578,...,2.827118,-0.530460,-2.604278,1.415303,-0.800768,-1.715997,0.180107,0.159367,-1.316734,-1.124902
1,-1.463956,-0.955104,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,1.398420,-1.278034,...,2.052320,-1.471858,-1.808852,1.449763,-0.837605,-1.458046,0.330237,0.527554,-0.397385,-1.054751
2,-1.263651,-0.812857,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,0.057571,-1.158785,...,1.026925,-2.998441,-1.021469,1.298615,-0.848825,-1.227711,0.402247,0.806003,0.268145,-0.713089
3,-1.058300,-0.830384,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,0.196419,-0.361698,...,0.440315,-0.354284,-0.509672,1.247129,-0.895700,-1.029691,0.567418,0.957081,0.654873,-0.208309
4,-0.847225,-0.923574,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,0.430422,1.272064,...,-0.162540,0.381997,0.288828,0.343841,-0.908780,-0.863954,0.780586,1.005557,0.800196,0.350216
5,-0.629747,-0.980599,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,0.038818,2.414964,...,-0.322787,0.895597,0.737950,-0.254598,-0.925120,-0.721075,0.948711,1.057989,0.886709,0.814700
6,-0.405189,-0.868902,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,-0.139605,1.657402,...,-0.360179,1.064850,0.848637,0.392261,-0.910964,-0.640511,1.130083,0.891357,0.531630,1.116973
7,-0.172871,-0.682303,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,0.089437,0.901517,...,-0.572689,0.769409,0.869060,0.866272,-0.929101,-0.188552,1.138875,0.953673,1.175773,1.341608
8,0.067883,-0.466005,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,0.151742,0.091818,...,-0.625681,0.751308,0.919747,0.407969,-0.935890,0.210381,0.953179,0.846349,1.469509,1.426900
9,0.223642,-0.070264,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,0.185184,-0.201511,...,-0.478547,0.729548,0.756669,-0.164930,-0.038401,0.531697,0.727660,0.496052,1.282377,1.394795


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     3092.530
1     3324.329
2     3665.682
3     4107.097
4     4300.676
5     3966.930
6     4199.911
7     4390.254
8     4733.632
9     4864.477
10    4454.468
11    3639.072
12    2811.648
13    2284.918
14    2391.212
15    2715.843
16    2935.607
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
17,-0.064841,1.959291,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.213454,...,-0.604115,0.147922,-0.308437,-1.75944,1.166651,0.74046,-1.172346,-1.219689,-1.234773,-0.016416


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    2829.682
Name: Rio De Janeiro - Consumo de Cimento (t), dtype: float64

In [12]:
train_input.iloc[-(len(train_input)//5):]

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
14,1.333259,1.260323,-1.195037,-0.553813,-0.504695,-0.765492,1.098789,0.71401,-1.705322,-0.139921,...,-0.708540,-0.181895,0.274027,-1.212778,1.370913,1.150765,-1.658434,-1.458301,-1.186853,-1.203759
15,1.162836,1.294615,-1.670834,-0.541784,-1.357653,0.164210,1.226007,0.54710,-1.702696,-0.105159,...,-0.657817,-0.593758,-0.368419,-1.431660,1.295665,1.170237,-1.626613,-1.265974,-0.810360,-0.575010
16,0.623127,1.547309,-0.776357,-0.721152,-1.478508,3.015562,1.346999,0.33274,0.058929,-0.152719,...,-0.650868,-0.442674,-0.700694,-1.722678,1.279383,1.028274,-1.514053,-1.285519,-1.005031,-0.242451


In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [14]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[777001412, 1827750373, 3448412777, 301263805, 2291538933, 3442215942, 4007924757, 282383594, 2192670847, 643693020, 45183756, 2270803603, 2494022293, 1300618014, 1190743723, 2585383451, 207991665, 73106657, 800705339, 2611573175, 538590540, 3755829537, 3225315732, 1948812051, 2899184898, 4108641708, 260301569, 1128291836, 3874196528, 3391013334, 1725050326, 3669426841, 2730380562, 597037187, 2955057187, 450306621, 1761013760, 1908787198, 726176036, 3338471032, 1266652594, 1115850394, 3175793885, 3356118721, 665619521, 84999461, 3907498537, 2325062576, 2341139404, 3934521124, 413949955, 2244736295, 3180769820, 1198021130, 1807034402, 715331263, 1670767915, 3418125149, 81232907, 839665128, 3462558541, 4182553840, 2117429972, 486582631, 815957956, 4007527189, 352105431, 159199359, 680887637, 494293249, 242389260, 2258580979, 1752105282, 436409222, 4030825247, 3234288889, 3501731066, 563315214, 4242327400, 869837362, 2339102179, 3043033771, 3245467472, 336536378, 1380316734, 1311864958, 3

val_loss: 35896.51953125


Step: 89 ___________________________________________
val_loss: 50005.28125


Step: 90 ___________________________________________
val_loss: 21869.333984375


Step: 91 ___________________________________________
val_loss: 41774.19921875


Step: 92 ___________________________________________
val_loss: 49699.83203125


Step: 93 ___________________________________________
val_loss: 50059.41796875


Step: 94 ___________________________________________
val_loss: 50008.0625


Step: 95 ___________________________________________
val_loss: 49770.140625


Step: 96 ___________________________________________
val_loss: 50326.23828125


Step: 97 ___________________________________________
val_loss: 50397.93359375


Step: 98 ___________________________________________
val_loss: 50748.26953125


Step: 99 ___________________________________________
val_loss: 50005.296875


final_seed: 3448412777


In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 459ms/step - loss: 13885586.0000 - val_loss: 6868502.5000
Epoch 2/10000
1/1 [==============================] - 0s 15ms/step - loss: 13494080.0000 - val_loss: 2169204.2500
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 13218446.0000 - val_loss: 13843848.0000
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 9052167.0000 - val_loss: 11805659.0000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 7509696.5000 - val_loss: 13764075.0000
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 8641337.0000 - val_loss: 12882275.0000
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 10306707.0000 - val_loss: 9303268.0000
Epoch 8/10000
1/1 [==============================] - 0s 14ms/step - loss: 9944159.0000 - val_loss: 11340327.0000
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 8592272.0000 - val_l

1/1 [==============================] - 0s 14ms/step - loss: 2578075.0000 - val_loss: 1654204.3750
Epoch 74/10000
1/1 [==============================] - 0s 15ms/step - loss: 2754614.0000 - val_loss: 1666796.0000
Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 2980063.0000 - val_loss: 1978710.6250
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 2587346.7500 - val_loss: 1533083.3750
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 2374980.7500 - val_loss: 1318665.3750
Epoch 78/10000
1/1 [==============================] - 0s 14ms/step - loss: 2547773.0000 - val_loss: 2634472.5000
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 3413077.5000 - val_loss: 1704517.5000
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 2664998.0000 - val_loss: 1453804.8750
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 2774986.0000 - val_loss: 2068129.000

1/1 [==============================] - 0s 14ms/step - loss: 1124030.6250 - val_loss: 379893.6250
Epoch 146/10000
1/1 [==============================] - 0s 14ms/step - loss: 1191828.1250 - val_loss: 853713.5000
Epoch 147/10000
1/1 [==============================] - 0s 15ms/step - loss: 1089472.0000 - val_loss: 591322.7500
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 1000174.3750 - val_loss: 531464.4375
Epoch 149/10000
1/1 [==============================] - 0s 15ms/step - loss: 917896.3125 - val_loss: 344928.7188
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 841893.8125 - val_loss: 416242.4688
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 776849.6250 - val_loss: 577233.0625
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 996734.0000 - val_loss: 426590.6562
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 894671.1875 - val_loss: 509435.0000
Epoc

1/1 [==============================] - 0s 14ms/step - loss: 526747.3750 - val_loss: 208367.1250
Epoch 219/10000
1/1 [==============================] - 0s 14ms/step - loss: 522976.5312 - val_loss: 199917.3750
Epoch 220/10000
1/1 [==============================] - 0s 14ms/step - loss: 435462.0000 - val_loss: 159299.1875
Epoch 221/10000
1/1 [==============================] - 0s 14ms/step - loss: 472797.5938 - val_loss: 177028.9219
Epoch 222/10000
1/1 [==============================] - 0s 14ms/step - loss: 423391.1250 - val_loss: 250891.6094
Epoch 223/10000
1/1 [==============================] - 0s 14ms/step - loss: 480045.8750 - val_loss: 117823.9766
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 508510.0625 - val_loss: 168075.9844
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 441225.5625 - val_loss: 195400.3125
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 577982.8750 - val_loss: 156354.0000
Epoch 22

1/1 [==============================] - 0s 14ms/step - loss: 260509.4375 - val_loss: 141274.1719
Epoch 292/10000
1/1 [==============================] - 0s 14ms/step - loss: 278750.5000 - val_loss: 143696.0625
Epoch 293/10000
1/1 [==============================] - 0s 14ms/step - loss: 232634.6406 - val_loss: 158146.1406
Epoch 294/10000
1/1 [==============================] - 0s 14ms/step - loss: 433024.3125 - val_loss: 445365.0000
Epoch 295/10000
1/1 [==============================] - 0s 14ms/step - loss: 595853.0625 - val_loss: 157019.1875
Epoch 296/10000
1/1 [==============================] - 0s 14ms/step - loss: 491755.7500 - val_loss: 176190.3281
Epoch 297/10000
1/1 [==============================] - 0s 14ms/step - loss: 411072.5938 - val_loss: 141602.2500
Epoch 298/10000
1/1 [==============================] - 0s 14ms/step - loss: 320686.8438 - val_loss: 149712.7031
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 194345.0312 - val_loss: 157776.0469
Epoch 30

...

Epoch 1099/10000
1/1 [==============================] - 0s 14ms/step - loss: 130326.7031 - val_loss: 180862.6719
Epoch 1100/10000
1/1 [==============================] - 0s 14ms/step - loss: 144357.8438 - val_loss: 492210.2500
Epoch 1101/10000
1/1 [==============================] - 0s 14ms/step - loss: 185116.0000 - val_loss: 353597.9688
Epoch 1102/10000
1/1 [==============================] - 0s 14ms/step - loss: 184036.5000 - val_loss: 344324.3750
Epoch 1103/10000
1/1 [==============================] - 0s 14ms/step - loss: 155910.8125 - val_loss: 345158.7812
Epoch 1104/10000
1/1 [==============================] - 0s 14ms/step - loss: 168766.9531 - val_loss: 253221.7969
Epoch 1105/10000
1/1 [==============================] - 0s 15ms/step - loss: 185204.1406 - val_loss: 213525.3594
Epoch 1106/10000
1/1 [==============================] - 0s 14ms/step - loss: 160804.0469 - val_loss: 246950.6406
Epoch 1107/10000
1/1 [==============================] - 0s 14ms/step - loss: 115467.1328 - val_l

1/1 [==============================] - 0s 15ms/step - loss: 182963.6094 - val_loss: 329483.6875
Epoch 1172/10000
1/1 [==============================] - 0s 14ms/step - loss: 133210.9062 - val_loss: 288474.4688
Epoch 1173/10000
1/1 [==============================] - 0s 15ms/step - loss: 77947.2969 - val_loss: 297734.8750
Epoch 1174/10000
1/1 [==============================] - 0s 14ms/step - loss: 138444.9219 - val_loss: 279452.6562
Epoch 1175/10000
1/1 [==============================] - 0s 14ms/step - loss: 74505.2812 - val_loss: 275875.9062
Epoch 1176/10000
1/1 [==============================] - 0s 14ms/step - loss: 151531.3594 - val_loss: 399773.2812
Epoch 1177/10000
1/1 [==============================] - 0s 14ms/step - loss: 127978.4688 - val_loss: 360773.7812
Epoch 1178/10000
1/1 [==============================] - 0s 14ms/step - loss: 138913.2188 - val_loss: 319305.5312
Epoch 1179/10000
1/1 [==============================] - 0s 14ms/step - loss: 116314.3516 - val_loss: 354504.4062
Ep

1/1 [==============================] - 0s 15ms/step - loss: 124169.6172 - val_loss: 192547.1250
Epoch 1244/10000
1/1 [==============================] - 0s 15ms/step - loss: 144659.2188 - val_loss: 164619.6875
Epoch 1245/10000
1/1 [==============================] - 0s 14ms/step - loss: 119965.1250 - val_loss: 173814.5938
Epoch 1246/10000
1/1 [==============================] - 0s 14ms/step - loss: 112973.3594 - val_loss: 173269.2344
Epoch 1247/10000
1/1 [==============================] - 0s 14ms/step - loss: 147025.0000 - val_loss: 142404.9531
Epoch 1248/10000
1/1 [==============================] - 0s 14ms/step - loss: 140108.3594 - val_loss: 167426.0156
Epoch 1249/10000
1/1 [==============================] - 0s 14ms/step - loss: 115903.1875 - val_loss: 127252.8359
Epoch 1250/10000
1/1 [==============================] - 0s 14ms/step - loss: 175457.4375 - val_loss: 141457.4844
Epoch 1251/10000
1/1 [==============================] - 0s 15ms/step - loss: 119486.9141 - val_loss: 179063.8750


1/1 [==============================] - 0s 14ms/step - loss: 145138.2344 - val_loss: 152875.6719
Epoch 1316/10000
1/1 [==============================] - 0s 14ms/step - loss: 119500.6484 - val_loss: 174802.8750
Epoch 1317/10000
1/1 [==============================] - 0s 14ms/step - loss: 118921.5781 - val_loss: 158690.8750
Epoch 1318/10000
1/1 [==============================] - 0s 14ms/step - loss: 114478.3438 - val_loss: 147266.2969
Epoch 1319/10000
1/1 [==============================] - 0s 14ms/step - loss: 123743.8828 - val_loss: 158673.1719
Epoch 1320/10000
1/1 [==============================] - 0s 14ms/step - loss: 151427.2188 - val_loss: 164657.2344
Epoch 1321/10000
1/1 [==============================] - 0s 14ms/step - loss: 135921.4375 - val_loss: 149022.7344
Epoch 1322/10000
1/1 [==============================] - 0s 14ms/step - loss: 97935.7500 - val_loss: 149041.9219
Epoch 1323/10000
1/1 [==============================] - 0s 14ms/step - loss: 109281.9766 - val_loss: 171047.1094
E

1/1 [==============================] - 0s 15ms/step - loss: 228844.4844 - val_loss: 254297.0625
Epoch 1388/10000
1/1 [==============================] - 0s 14ms/step - loss: 300378.2812 - val_loss: 236936.1875
Epoch 1389/10000
1/1 [==============================] - 0s 15ms/step - loss: 253241.5938 - val_loss: 207913.7344
Epoch 1390/10000
1/1 [==============================] - 0s 14ms/step - loss: 233288.5625 - val_loss: 198828.0156
Epoch 1391/10000
1/1 [==============================] - 0s 14ms/step - loss: 209477.9375 - val_loss: 185444.5781
Epoch 1392/10000
1/1 [==============================] - 0s 14ms/step - loss: 208897.4844 - val_loss: 220930.5469
Epoch 1393/10000
1/1 [==============================] - 0s 14ms/step - loss: 178361.5156 - val_loss: 222190.3750
Epoch 1394/10000
1/1 [==============================] - 0s 14ms/step - loss: 181299.1875 - val_loss: 295222.1250
Epoch 1395/10000
1/1 [==============================] - 0s 14ms/step - loss: 227471.6875 - val_loss: 224842.8281


In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 48ms/step
[2829.682](test_target) - [[2582.5425]](prediction) = 247.1395195312498


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
0,-1.485690,-2.292367,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,2.299346,-1.220147,...,1.885156,-0.191726,-1.786386,0.977618,1.848413,-1.616056,-1.458906,-2.229486,-2.156922,-1.297696
1,-1.082662,-0.183669,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,0.891536,-1.086586,...,1.224560,-0.913511,-1.141766,1.036671,1.011304,-1.049130,-1.024912,-0.937706,-0.949370,-1.221210
2,-0.670651,0.621648,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,-0.592240,-1.001662,...,0.350305,-2.083966,-0.503664,0.777652,0.756350,-0.542899,-0.816746,0.039227,-0.075205,-0.848693
3,-0.248261,0.522424,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,-0.438592,-0.434010,...,-0.149841,-0.056649,-0.088900,0.689422,-0.308873,-0.107689,-0.339276,0.569279,0.432757,-0.298325
4,0.185903,-0.005163,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,-0.179645,0.729486,...,-0.663838,0.507869,0.558211,-0.858522,-0.606118,0.256567,0.276944,0.739358,0.623637,0.310641
5,0.633236,-0.328003,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,-0.612992,1.543412,...,-0.800465,0.901654,0.922184,-1.884052,-0.977421,0.570587,0.762956,0.923316,0.737271,0.817073
6,1.095133,0.304359,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,-0.810435,1.003908,...,-0.832346,1.031424,1.011885,-0.775545,-0.655750,0.747652,1.287262,0.338688,0.270880,1.146644
7,1.572990,1.360772,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,-0.556978,0.465599,...,-1.013532,0.804904,1.028436,0.036756,-1.067906,1.740969,1.312678,0.557324,1.116953,1.391566


train_target:


,Rio De Janeiro - Consumo de Cimento (t)
0,3092.530
1,3324.329
2,3665.682
3,4107.097
4,4300.676
5,3966.930
6,4199.911
7,4390.254


test_input:


...

,Rio de Janeiro - IDH,Rio de-01eiro - Valor Cimento Portland,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rio de Janeiro - Desemprego,Rio de Janeiro - PIB - Estadual,Rio de Janeiro - PIB - Construção Civil,Rio de Janeiro - PIB - Per Capita,Rio de Janeiro - PIB Preços de Mercado,Rio De eiro-01 - Produção de Cimento (t)
17,-0.064841,1.959291,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,1.568753,-0.213454,...,-0.604115,0.147922,-0.308437,-1.75944,1.166651,0.74046,-1.172346,-1.219689,-1.234773,-0.016416


test_target:


,Rio De Janeiro - Consumo de Cimento (t)
17,2829.682


1/1 [==============================] - 0s 34ms/step
Error: 247.1395195312498




[4390.6181640625,
 4331.84375,
 4426.111328125,
 4560.228515625,
 3353.9169921875,
 3404.78564453125,
 2731.875,
 2399.988525390625,
 2631.5537109375,
 2582.54248046875]

In [21]:
display(targets)
display(predictions)

[4733.6320000000005,
 4864.477,
 4454.468,
 3639.072,
 2811.648,
 2284.918,
 2391.212,
 2715.843,
 2935.607,
 2829.682]

[4390.6181640625,
 4331.84375,
 4426.111328125,
 4560.228515625,
 3353.9169921875,
 3404.78564453125,
 2731.875,
 2399.988525390625,
 2631.5537109375,
 2582.54248046875]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

469.50071933593745

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.13948096326502998